## Overview
* Trained a model using the cropped images obtained from script crop right noise
* Increasing the size of data by doing augmentation is not yet implemented

# Import Libraries

In [42]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#Deep Learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers as L
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from skimage.transform import resize

import os
import cv2
import glob
import skimage.io as io
import pickle
from skimage.color import rgb2gray

#Data Visualizations
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [4]:
trainData = []
trainLabels = []

directory="/kaggle/input/croppeddata/cropped_characters/*/*"

paths = glob.glob(directory)
count=0
for path in paths:
    img = io.imread(path)
    imgBlurred = cv2.GaussianBlur(img, GAUSSIAN_SMOOTH_FILTER_SIZE, 0)
    imgThreshValue = cv2.threshold(imgBlurred, 0, 255, cv2.THRESH_OTSU)[1]
    imgPadded = sideBorder(imgThreshValue, 1)
    
    trainData.append(imgPadded)
    img_label = path.split('/')[-2]
    trainLabels.append(img_label)

trainData = np.array(trainData)
trainLabels = np.array(trainLabels)

print(trainLabels.shape)
print(trainData.shape)
print(trainData[0].shape)

In [ ]:
# Padding




# Blur

# Thresholding


In [5]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img

def change_size(image):
    img = array_to_img(image, scale=False) #returns PIL Image
    img = img.resize((150, 150)) #resize image
    arr = img_to_array(img) #convert back to array
    return arr.astype(np.float64)

## Apply augmentation
- rotation, shifts, brightness change

In [83]:
# The flow_from_directory() method allows you to read the images directly from the directory and augment them while the neural network model is learning on the training data.
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
#     zoom_range=0.2,
    rotation_range=30,
    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    '/kaggle/input/croppeddata/cropped_characters',
    target_size=(150, 150),
    batch_size=128,
    class_mode='categorical',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    '/kaggle/input/croppeddata/cropped_characters', # same directory as training data
    target_size=(150, 150),
    batch_size=20,
    class_mode='categorical',
    subset='validation') # set as validation data

We split the data into the train and validation sets. Here is the distribution of the split data.

In [16]:
sns.barplot(['train', 'valid'], [train_generator.n, validation_generator.n])

In [84]:
# model = Sequential()

# model.add(tf.keras.applications.resnet50.ResNet50(input_shape = (150, 150, 3), 
#                                 include_top = False, 
#                                 weights = 'imagenet'))

# model.add(L.Flatten())
# model.add(tf.keras.layers.Dropout(0.2))
# model.add(L.Dense(256, activation='relu'))
# model.add(tf.keras.layers.Dropout(0.2))
# model.add(L.Dense(128, activation='relu'))
# model.add(L.Dense(38, activation='softmax'))

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2), 
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'), 
    tf.keras.layers.MaxPooling2D(2,2),
    
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'), 
    
    tf.keras.layers.Dense(38, activation='softmax')  
])

model.compile(optimizer=keras.optimizers.Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
#Do not use default learning rate since it is too high!

In [93]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        # Check accuracy
        if(logs.get('accuracy') > 90):
            # Stop if threshold is met
            print("\naccuracy of training is bigger than 90!")
            self.model.stop_training = True

# Instantiate class
callbacks = myCallback()

In [86]:
# for layer in model.layers[0].layers:
#     if layer.name == 'conv5_block1_0_conv':
#         break
#     layer.trainable=False

In [87]:
history = model.fit(train_generator, epochs=20, 
            validation_data=validation_generator,
            steps_per_epoch=train_generator.n//train_generator.batch_size,
            validation_steps=validation_generator.n//validation_generator.batch_size)

In [88]:
#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc      = history.history[     'accuracy' ]
val_acc  = history.history[ 'val_accuracy' ]
loss     = history.history[    'loss' ]
val_loss = history.history['val_loss' ]

epochs   = range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot  ( epochs,     acc )
plt.plot  ( epochs, val_acc )
plt.title ('Training and validation accuracy')
plt.legend(['train', 'test'])
plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot  ( epochs,     loss )
plt.plot  ( epochs, val_loss )
plt.title ('Training and validation loss'   )
plt.legend(['train', 'test'])

In [89]:
# save the model to disk
dirpath = 'char_detect_model'

model.save('char_detect_model')

In [94]:
model = keras.models.load_model('char_detect_model')
model.summary()

In [ ]:
history = model.fit(train_generator, epochs=40, 
            validation_data=validation_generator,
            steps_per_epoch=train_generator.n//train_generator.batch_size,
            validation_steps=validation_generator.n//validation_generator.batch_size, callbacks=[callbacks])

In [57]:
labels = os.listdir('../input/croppeddata/cropped_characters')

def predict(img, model, labels):
    img = resize(img, (150,150,3), anti_aliasing=True)
    img = np.expand_dims(img, axis=0)
    
    preds = model.predict(img)
    return labels[np.argmax(preds)]

In [44]:
print(labels)

In [18]:
#!pip install imutils

In [59]:
#import pytesseract
from imutils import paths
import argparse
import imutils
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from skimage.segmentation import clear_border
import numpy as np
import skimage 
from PIL import Image
#from imutils.perspective import four_point_transform
%matplotlib inline

In [70]:
# module level variables ##########################################################################
GAUSSIAN_SMOOTH_FILTER_SIZE = (5, 5)
ADAPTIVE_THRESH_BLOCK_SIZE = 19
ADAPTIVE_THRESH_WEIGHT = 9

###################################################################################################
def preprocess(imgOriginal):
    imgSaturation, imgGrayscale = extractValue(imgOriginal)

    imgMaxContrastGrayscale = maximizeContrast(imgGrayscale)

    imgBlurred = cv2.GaussianBlur(imgMaxContrastGrayscale, GAUSSIAN_SMOOTH_FILTER_SIZE, 0)

    imgThreshValue = cv2.threshold(imgBlurred, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    imgThreshSaturation = cv2.threshold(imgSaturation, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

    # plt.imshow(imgBlurred,cmap='gray')
    # plt.show()
    # plt.imshow(imgThreshValue,cmap='gray')
    # plt.show()

    # plt.imshow(imgSaturation,cmap='gray')
    # plt.show()
    # plt.imshow(imgThreshSaturation,cmap='gray')
    # plt.show()

    #imgThresh = cv2.adaptiveThreshold(imgBlurred, 255.0, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, ADAPTIVE_THRESH_BLOCK_SIZE, ADAPTIVE_THRESH_WEIGHT)
    
    return imgGrayscale, (imgThreshValue & (255 - imgThreshSaturation))
# end function

def preprocess2(imgOriginal):
    _, imgGrayscale = extractValue(imgOriginal)

    imgBlurred = cv2.GaussianBlur(imgGrayscale, GAUSSIAN_SMOOTH_FILTER_SIZE, 0)

    imgThreshValue = cv2.threshold(imgBlurred, 0, 255, cv2.THRESH_OTSU)[1]

    # plt.imshow(imgBlurred,cmap='gray')
    # plt.show()
    # plt.imshow(imgThreshValue,cmap='gray')
    # plt.show()

    # plt.imshow(imgSaturation,cmap='gray')
    # plt.show()
    # plt.imshow(imgThreshSaturation,cmap='gray')
    # plt.show()

    #imgThresh = cv2.adaptiveThreshold(imgBlurred, 255.0, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, ADAPTIVE_THRESH_BLOCK_SIZE, ADAPTIVE_THRESH_WEIGHT)
    
    return imgThreshValue
# end function

###################################################################################################
def extractValue(imgOriginal):
    height, width, numChannels = imgOriginal.shape

    imgHSV = np.zeros((height, width, 3), np.uint8)

    imgHSV = cv2.cvtColor(imgOriginal, cv2.COLOR_BGR2HSV)

    imgHue, imgSaturation, imgValue = cv2.split(imgHSV)

    return imgSaturation, imgValue
# end function

###################################################################################################
def maximizeContrast(imgGrayscale):

    height, width = imgGrayscale.shape

    imgTopHat = np.zeros((height, width, 1), np.uint8)
    imgBlackHat = np.zeros((height, width, 1), np.uint8)

    structuringElement = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))

    imgTopHat = cv2.morphologyEx(imgGrayscale, cv2.MORPH_TOPHAT, structuringElement)
    imgBlackHat = cv2.morphologyEx(imgGrayscale, cv2.MORPH_BLACKHAT, structuringElement)

    imgGrayscalePlusTopHat = cv2.add(imgGrayscale, imgTopHat)
    imgGrayscalePlusTopHatMinusBlackHat = cv2.subtract(imgGrayscalePlusTopHat, imgBlackHat)

    return imgGrayscalePlusTopHatMinusBlackHat
# end function

In [71]:
def order_points(pts):
	# initialzie a list of coordinates that will be ordered
	# such that the first entry in the list is the top-left,
	# the second entry is the top-right, the third is the
	# bottom-right, and the fourth is the bottom-left
	rect = np.zeros((4, 2), dtype = "float32")
	# the top-left point will have the smallest sum, whereas
	# the bottom-right point will have the largest sum
	s = pts.sum(axis = 1)
	rect[0] = pts[np.argmin(s)]
	rect[2] = pts[np.argmax(s)]
	# now, compute the difference between the points, the
	# top-right point will have the smallest difference,
	# whereas the bottom-left will have the largest difference
	diff = np.diff(pts, axis = 1)
	rect[1] = pts[np.argmin(diff)]
	rect[3] = pts[np.argmax(diff)]
	# return the ordered coordinates
	return rect

In [73]:
def four_point_transform(image, pts):
	# obtain a consistent order of the points and unpack them
	# individually
	rect = order_points(pts)
	(tl, tr, br, bl) = rect
	# compute the width of the new image, which will be the
	# maximum distance between bottom-right and bottom-left
	# x-coordiates or the top-right and top-left x-coordinates
	widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
	widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
	maxWidth = max(int(widthA), int(widthB))
	# compute the height of the new image, which will be the
	# maximum distance between the top-right and bottom-right
	# y-coordinates or the top-left and bottom-left y-coordinates
	heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
	heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
	maxHeight = max(int(heightA), int(heightB))
	# now that we have the dimensions of the new image, construct
	# the set of destination points to obtain a "birds eye view",
	# (i.e. top-down view) of the image, again specifying points
	# in the top-left, top-right, bottom-right, and bottom-left
	# order
	dst = np.array([
		[0, 0],
		[maxWidth - 1, 0],
		[maxWidth - 1, maxHeight - 1],
		[0, maxHeight - 1]], dtype = "float32")
	# compute the perspective transform matrix and then apply it
	M = cv2.getPerspectiveTransform(rect, dst)
	warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
	# return the warped image
	return warped

In [74]:
def contourApprox(imgOriginal, cnts):
	# initialize a contour that corresponds to the receipt outline
	receiptCnt = []
	firstContour = False
	firstArea = 0
	firstPeri = 0

	# loop over the contours
	for c in cnts:
		# approximate the contour
		peri = cv2.arcLength(c, True) + 0.01
		approx = cv2.approxPolyDP(c, 0.05 * peri, True)

		area = cv2.contourArea(approx) + 0.01

		drawnContour = imgOriginal.copy()
		cv2.drawContours(drawnContour, [approx], -1, (0, 255, 0), 2)
# 		plt.imshow(drawnContour)
# 		plt.show()
		
		if area < 3000:
			break

		# if our approximated contour has four points, then we can
		# assume we have found the outline of the receipt
		if len(approx) == 4 and (not firstContour or (firstPeri / peri < 1.4 and firstPeri / peri > 0.7 and firstArea / area < 1.4 and firstArea / area > 0.7)):
			receiptCnt.append(approx)
			if firstContour:
				 break
			firstContour = True
			firstArea = area
			firstPeri = peri
			
		else:
			approx = cv2.convexHull(approx)
			peri = cv2.arcLength(approx, True) + 0.01
			area = cv2.contourArea(approx) + 0.01
			if len(approx) == 4 and (not firstContour or (firstPeri / peri < 1.4 and firstPeri / peri > 0.7 and firstArea / area < 1.4 and firstArea / area > 0.7)):
				receiptCnt.append(approx)
				if firstContour:
					break
				firstContour = True
				firstArea = area
				firstPeri = peri
				break		
	# if the receipt contour is empty then our script could not find the
	# outline and we should be notified
	return receiptCnt

In [75]:
def showApprox(imgOriginal, thresh, receiptCnt):
    cv2.drawContours(imgOriginal, [receiptCnt], -1, (0, 255, 0), 2)
    #print(output[1])
    plt.imshow(imgOriginal)
    plt.show()
    rect_coor = np.array(receiptCnt.reshape(4,2))
    img = four_point_transform(imgOriginal, rect_coor)
    #print(img.shape)
    #img_new = add_margin(img, 5, 10, 0, 10)
    final_img = preprocess2(img)
#     plt.imshow(final_img, cmap="gray")
#     plt.show()
    return (final_img,img)
    #print(f"boundaries = {receiptCnt.reshape(4, 2)}")


In [76]:
def add_margin(pil_img, top, right, bottom, left, color=(255,255,255)):
    height,width,_ = pil_img.shape
    new_width = width + right + left
    new_height = height + top + bottom
    result = Image.new('RGB', (new_width, new_height), color)
    result.paste(pil_img, (left, top))
    return result

In [77]:
def CutLetters(img,rgb_img, count):
    height, width = img.shape
    windowWidthR = int(0.18 * width * count)
    windowWidthL = int(0.12 * width * count)
    windowWidth = windowWidthR
    if count == 2:
        windowWidthR = windowWidthL
    startRatio = int(0.1 * windowWidth)
    windowHeight = img.shape[0]
    marginWidth = int(windowWidth * 0.05)
    stepSize = 8
    GroupSize = 8
    lowerBlackLimit = 0.07
    upperBlackLimit = 0.9

    Letters = []
    Letters_imgs = []
    Letters_imgs_thresh = []
    for i in range(0, width - (windowWidth), stepSize * GroupSize):

        Group = []
        for j in range(i, min(i + stepSize * GroupSize, width - (windowWidth)), stepSize):
            #plt.imshow(img[:, j:j+windowWidth],cmap='gray')
            #plt.show()
            if j <= width/2:
                windowWidth = windowWidthL
            else:
                windowWidth = windowWidthR

            blackCountMarginL = marginWidth * height  - np.count_nonzero(img[:, j : j+marginWidth])
            blackCountMarginR = marginWidth * height  - np.count_nonzero(img[:, j+windowWidth-marginWidth : j+windowWidth])
            blackCountInner = (windowWidth - 2 * startRatio) * height - np.count_nonzero(img[:, j+startRatio : j+windowWidth-startRatio])
            
            #print(blackCountMarginL, marginWidth * height)
            #print(blackCountMarginR, marginWidth * height)
            #print(blackCountInner, (windowWidth - 2 * marginWidth) * height)
            #and blackCountMarginR < 0.2 * marginWidth * height \

            if blackCountMarginL < 0.17 * marginWidth * height and blackCountMarginR < 0.17 * marginWidth * height and blackCountInner > lowerBlackLimit * (windowWidth - 2 * startRatio) * height \
                and blackCountInner < upperBlackLimit * (windowWidth - 2 * startRatio) * height:
                Group.append((blackCountInner, j))
        if len(Group) > 0:
            max_G = max(Group)[1]
            if len(Letters) == 0 or len(Letters) > 0 and max_G - Letters[-1] > 21:
                Letters.append(max_G)
              # rect_coord = [rgb_img[0][0],rgb_img[0][-1],max(Group)[1],max(Group)[1]+windowWidth]
              # rect_coord = np.array(rect_coord,dtype='int32')
                rgb2_img = rgb_img[:, max_G:max_G+windowWidth]
                Letters_imgs.append(rgb2_img)
                #               plt.imshow(rgb2_img)
                thresh_img = img[:, max_G:max_G+windowWidth]
                Letters_imgs_thresh.append(thresh_img)
              #plt.imshow(img[:, max(Group)[1]:max(Group)[1]+windowWidth],cmap='gray')
#               plt.show()


    return Letters_imgs,Letters_imgs_thresh






In [78]:
def sideBorder(img, d):
    h,w=img.shape[0:2]
    base_size=h+20,w+20,3
    if d == 1:
        base_size=h+20,w+20
    # make a 3 channel image for base which is slightly larger than target img
    base=np.zeros(base_size,dtype=np.uint8)
    if d == 1:
        cv2.rectangle(base,(0,0),(w+20,h+20),255,20) # really thick white rectangle
    else:
        cv2.rectangle(base,(0,0),(w+20,h+20),(255,255,255),20) # really thick white rectangle
    
    base[10:h+10,10:w+10]=img # this works
    return base

In [80]:
## Trying to get letters from cropped image, not working yet
def CharacterContours(thresh_img):
    cnts2 = cv2.findContours(thresh_img.copy(), cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE)
    cnts2 = imutils.grab_contours(cnts2)
    cnts2 = sorted(cnts2, key=cv2.contourArea, reverse=True)
    #cv2.drawContours(thresh_img, cnts, -1, (0, 255, 0), 2)
    #plt.imshow(thresh_img, cmap="gray")
    #plt.show()

    letters = []
    letters_adj = []
    for c in cnts2:
        # approximate the contour
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.1 * peri, True)
        letters.append(approx)
    #print(letters)
    #print(len(cnts))
    #print(len(letters))
    # for i in range(len(letters)):
    #     # print(i)
    #     arr = np.array(letters[i],dtype='int32')
    #     # arr.reshape(4,2)
    #     arr = arr.reshape(len(arr[:]),2)
    #     letters_adj.append(arr)
    #print(letters_adj[1].reshape(len(letters_adj[1][:]),2))
    # print(letters_adj[2])
    # output2 = final_img.copy()
    # cv2.drawContours(output2, [letters_adj[6]], -1, (0, 255, 0), 2)
    # plt.imshow(output2)
    # plt.show()
    # letters_img = four_point_transform(final_img, letters_adj[1])
    # plt.imshow(letters_img)
    # plt.show()



In [82]:
for i in range(30):
    imgOriginal = cv2.imread(f"../input/testdata/test_images/{str(i).zfill(5)}.jpg")
    #imgOriginal = cv2.imread("./train_images/02937.jpg")
    #img = imgOriginal.copy()
    imgOriginal = imutils.resize(imgOriginal, width=500)
    #ratio = img.shape[1] / float(imgOriginal.shape[1])
#     plt.imshow(imgOriginal)
#     plt.show()
    gray,thresh = preprocess(imgOriginal)
    # plt.imshow(gray,cmap='gray')
    # plt.show()
#     plt.imshow(thresh,cmap='gray')
#     plt.show()

    #ClosedThresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_RECT, (15, 3)))
    OpenedThresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_RECT, (7, 7)))

    minValue = np.min(OpenedThresh)

    OpenedThresh[imgOriginal.shape[0]-4:imgOriginal.shape[0]-1, :] = minValue
    OpenedThresh[:, imgOriginal.shape[1]-4:imgOriginal.shape[1]-1] = minValue
    OpenedThresh[:, 0:3] = minValue


#     plt.imshow(OpenedThresh,cmap='gray')
#     plt.show()



    #plt.imshow(ClosedThresh,cmap='gray')
    #plt.show()
    

    cnts = cv2.findContours(OpenedThresh.copy(), cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
    # for c in cnts:
        # #print(c[0])
        # plt.scatter(c[:, 0, 0], -c[:, 0, 1])
        # plt.show()
    # np.array(cnts,dtype=np.int32).shape
    #print(cnts[-1])
    receiptCnt = contourApprox(imgOriginal, cnts)
    if len(receiptCnt) == 0:
        
        receiptCnt = [np.array([[[0,  0]], [[0, imgOriginal.shape[0]]],
         [[imgOriginal.shape[1], imgOriginal.shape[0]]], [[imgOriginal.shape[1], 0]]])]
    #print(receiptCnt)

    for r in receiptCnt:

        final_img ,rgb_img= showApprox(imgOriginal, thresh, r)
        final_img = sideBorder(final_img, 1)
        rgb_img = sideBorder(rgb_img, 3)
#         plt.imshow(rgb_img)
#         plt.show()
#         plt.imshow(final_img,cmap='gray')
#         plt.show()
        #resized_img = imutils.resize(final_img[:,:int(final_img.shape[1]/2)], width=int(1.5 * final_img.shape[1]/2))
        #new_img = np.concatenate((resized_img,final_img[:,int(final_img.shape[1]/2):]),axis=1)
        #print(f"index = {int(final_img.shape[1]/2)}")

        #print(resized_img.shape[1])
        # new_im = Image.new('RGB', (resized_img.shape[0]+final_img[:,int(final_img.shape[1]/2):].shape[1],
        # max(resized_img.shape[0],final_img.shape[0])), (250,250,250))

        # new_im.paste(resized_img, (0,0))
        # new_im.paste(final_img[:,int(final_img.shape[1]/2):], (resized_img.shape[1],0))
        # plt.imshow(new_im)
        # plt.show()
        #CutLetters(rgb_img)

        letters_cropped, letters_cropped_thresh = CutLetters(final_img,rgb_img, len(receiptCnt))
        for character in letters_cropped_thresh:
            charcater = np.array(character)
            plt.imshow(charcater)
            plt.show()
            print(predict(img=character, model=model, labels=labels))
          #print(final_img.shape[1])
          #CharacterContours(final_img)

# plt.plot(cnts[0])
# plt.show()

#cnts = skimage.measure.find_contours(thresh)
#cnts = sorted(cnts, reverse=True)
#cnts

# for c in cnts:
#     # compute the bounding box of the contour and then use
#     # the bounding box to derive the aspect ratio
#     (x, y, w, h) = cv2.boundingRect(c)
#     ar = w / float(h)
#     if ar >= 2 and ar <= 10:
#         lpCnt = c
#         letter = gray[y:y + h, x:x + w]
#         roi = cv2.threshold(letter, 0, 255,
#                 cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
#             # check to see if we should clear any foreground
#             # pixels touching the border of the image
#             # (which typically, not but always, indicates noise)
#         # display any debugging information and then break
#         # from the loop early since we have found the license
#         # plate region
#         # plt.plot(letter,cmap='gray')
#         # plt.show()
#         plt.plot(roi,cmap='gray')
#         plt.show()
#         print("****************")
# print(roi)   